RAIN - Real & Artificial Intelligence for Neuroscience

## Video handling

Welcome to my video handling notebook!

Here you'll find are a compilation of many functions I've written when working with videos.

Since we record behavioral experiments, sometimes it is useful to cut, reshape, draw on, and add text to videos, among other things.

In [1]:
import os
import numpy as np

import cv2
from tkinter import Tk, filedialog, messagebox
from PyQt5 import QtWidgets, QtCore

---
### The most common video editing tasks are:
- Resize the video if it is too big (heavy)
- Crop the video if it is too long

In [ ]:
def select_video_files():
    """Opens a file dialog to select multiple video files."""
    options = QtWidgets.QFileDialog.Options()
    files, _ = QtWidgets.QFileDialog.getOpenFileNames(None, "Select Video Files", "", "Video Files (*.mp4 *.avi *.mov);;All Files (*)", options=options)
    return files

def get_video_info(file_path):
    """Gets the resolution of a video file."""
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        return None, None
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    return width, height, fps

def change_video_resolution(file_path, width, height, fps, start_time=0, end_time=None):
    """
    Changes the resolution of a video and optionally trims it.
    
    Parameters:
    - file_path: Path to the input video file.
    - width: Target width of the video.
    - height: Target height of the video.
    - start_time: Start time in seconds for the output video.
    - end_time: End time in seconds for the output video (None for full length).
    """

    folder_path = os.path.dirname(file_path)
    output_folder = os.path.join(folder_path, "Resized")
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(file_path))

    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return
        
    # Get the original video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    # Convert time to frame numbers
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps) if end_time else total_frames
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = start_frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count >= end_frame:
            break
        
        # Resize the frame
        resized_frame = cv2.resize(frame, (width, height))

        # Write the frame to the output video
        out.write(resized_frame)
        frame_count += 1

    cap.release()
    out.release()
    print(f"Video saved to {output_path} with resolution {width}x{height}, cropped from {start_time}s to {end_time if end_time else 'end'}s")

class VideoEditor(QtWidgets.QWidget):
    def __init__(self, video_files):
        super().__init__()
        self.video_files = video_files
        self.current_file = video_files[0] if video_files else None
        
        # Get initial resolution
        self.original_width, self.original_height, self.original_fps = get_video_info(self.current_file)
        
        # Set up the GUI
        self.setWindowTitle('Video Edit') 
        self.resize(400, 200)

        layout = QtWidgets.QVBoxLayout()
        
        self.start_label = QtWidgets.QLabel('Enter start & end time:')
        self.start_edit = QtWidgets.QTimeEdit()
        self.start_edit.setDisplayFormat("mm:ss")
        self.start_edit.setTime(QtCore.QTime(0, 0))
        
        self.end_edit = QtWidgets.QTimeEdit()
        self.end_edit.setDisplayFormat("mm:ss")
        self.end_edit.setTime(QtCore.QTime(0, 5))
        
        self.res_label = QtWidgets.QLabel(f'Enter resolution (default {self.original_width}x{self.original_height}):')
        self.width_edit = QtWidgets.QSpinBox()
        self.width_edit.setRange(100, 4000)
        self.width_edit.setValue(self.original_width)
        
        self.height_edit = QtWidgets.QSpinBox()
        self.height_edit.setRange(100, 4000)
        self.height_edit.setValue(self.original_height)

        # Maintain aspect ratio
        self.width_edit.valueChanged.connect(self.update_height)
        self.height_edit.valueChanged.connect(self.update_width)

        self.fps_label = QtWidgets.QLabel(f'Enter fps (default {self.original_fps}):')
        self.fps_edit = QtWidgets.QSpinBox()
        self.fps_edit.setRange(1, 120)
        self.fps_edit.setValue(self.original_fps)
        
        self.run_button = QtWidgets.QPushButton('Run')
        self.run_button.clicked.connect(self.run)
        
        layout.addWidget(self.start_label)
        layout.addWidget(self.start_edit)
        layout.addWidget(self.end_edit)
        layout.addWidget(self.res_label)
        layout.addWidget(self.width_edit)
        layout.addWidget(self.height_edit)
        layout.addWidget(self.fps_label)
        layout.addWidget(self.fps_edit)
        layout.addWidget(self.run_button)
        
        self.setLayout(layout)

    def update_height(self):
        aspect_ratio = self.original_height / self.original_width
        self.height_edit.blockSignals(True)
        self.height_edit.setValue(int(self.width_edit.value() * aspect_ratio))
        self.height_edit.blockSignals(False)

    def update_width(self):
        aspect_ratio = self.original_width / self.original_height
        self.width_edit.blockSignals(True)
        self.width_edit.setValue(int(self.height_edit.value() * aspect_ratio))
        self.width_edit.blockSignals(False)

    def run(self):
        start_time = self.start_edit.time().minute() * 60 + self.start_edit.time().second()
        end_time = self.end_edit.time().minute() * 60 + self.end_edit.time().second()
        width = self.width_edit.value()
        height = self.height_edit.value()
        fps = self.fps_edit.value()
        
        for file in self.video_files:
            change_video_resolution(file, width, height, fps, start_time, end_time)
        
        print("Processing complete.")
        self.close()  # Automatically close the GUI after processing

In [3]:
if __name__ == '__main__':
    import sys
    app = QtWidgets.QApplication(sys.argv)
    video_files = select_video_files()
    if video_files:
        window = VideoEditor(video_files)
        window.show()
        sys.exit(app.exec_())

---
### The following function will let you crop the video to the desired size.
It's useful for when you have unused space in the video, or if you record multiple animals simultaneously with one camera.

In [11]:
def select_video_files() -> list:
    """Select video files using a file dialog.

    Returns:
        list: List of selected video files.
    """
    import tkinter.filedialog as filedialog
    from tkinter import Tk

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()
    
    # Open file dialog to select video files
    video_files = filedialog.askopenfilenames(
        title="Select Video Files",
        filetypes=[("Video Files", "*.mp4 *.avi *.mkv *.mov")]
    )
    if not video_files:
        raise ValueError("No video files selected.")
    
    print(f"Selected {len(video_files)} videos.")

    return video_files

def merge_frames(video_files: list) -> np.ndarray:
    """
    Merge the first frame of each video file into a single image.

    Args:
        video_files (list): List of video files.
    
    Returns:
        np.ndarray: Merged image.
    """
    merged_image = None
    
    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        success, frame = cap.read()
        cap.release()
        
        if not success:
            print(f"Could not read first frame of {video_file}")
            continue
        
        # Calculate transparency
        transparency = round(1 / len(video_files), 4)
        transparent_frame = (frame * transparency).astype(np.uint8)
        
        if merged_image is None:
            # Initialize merged image
            merged_image = np.zeros_like(transparent_frame)
        
        # Add transparent frame to the merged image
        merged_image = cv2.add(merged_image, transparent_frame)
    
    return merged_image

def define_rectangle(x1, y1, x2, y2):
    """Define a rectangle based on two points.
    """
    width = int(round(abs(x2 - x1)))
    height = int(round(abs(y2 - y1)))
    center = [int(round((x1+x2)//2)), int(round((y1+y2)//2))] # Round to integer
    return center, width, height

def draw_rectangle(image, center, width, height, angle = 0, color = (0, 255, 0), thickness = 2):
    """Draw a rectangle on an image.
    """
    box = cv2.boxPoints(((center[0], center[1]), (width, height), angle))
    box = np.intp(box)  # Convert to integer
    cv2.drawContours(image, [box], 0, color, thickness)
    cv2.circle(image, (int(round(center[0])), int(round(center[1]))), radius=2, color=color, thickness=-1)

In [12]:
# Function to crop a video
def crop_video(file_path, center, width, height, angle = 0):
    """Crop a video to a specific size and center.

    Args:
        file_path (str): Path to the video file.
        center (tuple): Center of the cropped region.
        width (int): Width of the cropped region.
        height (int): Height of the cropped region.
        angle (int, optional): Angle of rotation. Defaults to 0.
    """
    
    folder_path = os.path.dirname(file_path)
    output_folder = os.path.join(folder_path, "cropped")
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(file_path))
    
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return
        
    # Output video writer with the cropped size
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if angle != 0:
            # Rotate the entire frame
            rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
            frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))
        
        # Crop the rotated region
        frame = frame[
            center[1] - height // 2:center[1] + height // 2,
            center[0] - width // 2:center[0] + width // 2,
        ]
        out.write(frame)

    cap.release()
    out.release()
    print(f"Cropped video saved to {output_path} with size {width}x{height}.")

def cropper():

    # Get video files
    video_files = select_video_files()

    # Merge frames
    image = merge_frames(video_files)
    
    # Get original dimensions and print them
    width = image.shape[1]
    height = image.shape[0]
    print(f"Original Size: {width}x{height}")

    # Initialize variables
    clone = image.copy()
    corners = []  # Current ROI 
    dragging = [False]  # For moving ROI
    drag_start = None
    angle = 0  # Current angle for rotation
    rotate_factor = 1  # Amount of change per scroll
    resize_factor = 2  # Amount of change per scroll
    scale_factor = 1.0  # Scaling factor
    square = False  # For enforcing a square shape

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal clone, corners, dragging,drag_start, angle, rotate_factor, resize_factor, square

        # Adjust mouse coordinates according to scale factor
        x = int(x / scale_factor)
        y = int(y / scale_factor)

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            # angle = 0 # Reset angle
            if not dragging[0]:
                dragging[0] = True
                corners = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 1:
            x1, y1 = corners[0]
            x2, y2 = x, y # While dragging, update the end point

            # If Ctrl is held, enforce a square shape
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                side = max(abs(x2 - x1), abs(y2 - y1))
                x2 = x1 + side if x2 > x1 else x1 - side
                y2 = y1 + side if y2 > y1 else y1 - side
                square = True
            else:
                square = False

            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                x1, y1 = corners[0]  # Start point
                x2, y2 = x, y  # End point
                if square:
                    side = max(abs(x2 - x1), abs(y2 - y1))
                    x2 = x1 + side if x2 > x1 else x1 - side
                    y2 = y1 + side if y2 > y1 else y1 - side
                corners.append((x2, y2))
                
        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(corners) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            x1, y1, x2, y2 = (corners[0][0] + dx, corners[0][1] + dy, corners[1][0] + dx, corners[1][1] + dy)
            corners[0] = x1, y1
            corners[1] = x2, y2
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(corners) == 2:
            dragging[0] = False

        # Resize or rotate the ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(corners) == 2:
            x1, y1 = corners[0]
            x2, y2 = corners[1]
            if flags & cv2.EVENT_FLAG_CTRLKEY:  # Rotate with Ctrl key pressed
                if flags > 0:  # Scroll up
                    angle -= rotate_factor
                else:  # Scroll down
                    angle += rotate_factor
            else:  # Resize without modifier key
                width = max(abs(x2 - x1), 1)
                height = max(abs(y2 - y1), 1)
                ratio = width/height
                if flags > 0:  # Scroll up
                    x1 -= resize_factor*ratio
                    y1 -= resize_factor
                    x2 += resize_factor*ratio
                    y2 += resize_factor
                else:  # Scroll down
                    x1 += resize_factor*ratio
                    y1 += resize_factor
                    x2 -= resize_factor*ratio
                    y2 -= resize_factor
                corners = [(x1, y1), (x2, y2)]
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Draw the updated ROI and display width, height, and angle
        if len(corners) > 0:
            x1, y1 = corners[0]
            if len(corners) > 1:
                x2, y2 = corners[1]
            else:
                x2, y2 = x, y
                if square:
                    side = max(abs(x2 - x1), abs(y2 - y1))
                    x2 = x1 + side if x2 > x1 else x1 - side
                    y2 = y1 + side if y2 > y1 else y1 - side
            
            # Display height, width, and angle at the bottom of the frame
            center, width, height = define_rectangle(x1, y1, x2, y2)
            text = f"M: [{x}, {y}], C: {center}, W: {width}, H: {height}, A: {angle}"  # Convert to int for display
        else:
            text = f"M: [{x}, {y}]"

        font_scale, font_thickness = 0.5, 1
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        text_x, text_y = 10, clone.shape[0] - 10
        cv2.rectangle(clone, (text_x - 5, text_y - text_size[1] - 5), 
                    (text_x + text_size[0] + 8, text_y + 5), (0, 0, 0), -1)  # Background for text
        cv2.putText(clone, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (255, 255, 255), font_thickness)

    # Set up the OpenCV window and bind the mouse callback
    cv2.namedWindow("Select Region")
    cv2.setMouseCallback("Select Region", handle_mouse)

    while True:
        display_image = cv2.resize(clone, None, fx=scale_factor, fy=scale_factor)
        cv2.imshow("Select Region", display_image)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('+') and scale_factor < 2:
            scale_factor += 0.1  # Zoom in
        elif key == ord('-') and scale_factor > 0.5:
            scale_factor -= 0.1  # Zoom out
        elif key == ord('r'):
            scale_factor = 1.0  # Reset zoom
        

        elif key == ord('c') and len(corners) == 2:  # Save the ROI
            response = messagebox.askquestion("Crop", "Do you want to crop this region?")
            if response == 'yes':
                break

        elif key == ord('q'):  # Quit and save
            response = messagebox.askquestion("Exit", "Do you want to exit the cropper?")
            if response == 'yes':
                print("Cropping canceled.")
                cv2.destroyAllWindows()
                return                

    cv2.destroyAllWindows()

    # Ensure valid ROI
    x1, y1 = corners[0]
    x2, y2 = corners[1]
    center, width, height = define_rectangle(x1, y1, x2, y2)

    # Apply cropping
    for file in video_files:
        crop_video(file, center, width, height, angle)

    print(f"Video cropped and saved")

In [13]:
# Call the function
cropper()

Selected 18 videos.
Original Size: 1600x900
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned\cropped\WIN_20240405_13_49_18_Pro_aligned.mp4 with size 890x890.
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned\cropped\WIN_20240405_13_56_38_Pro_aligned.mp4 with size 890x890.
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned\cropped\WIN_20240405_14_04_19_Pro_aligned.mp4 with size 890x890.
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned\cropped\WIN_20240405_14_12_24_Pro_aligned.mp4 with size 890x890.
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned\cropped\WIN_20240405_14_19_42_Pro_aligned.mp4 with size 890x890.
Cropped video saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Ali